# Nate's guide to speeding up dfs in floops!

For this tutorial, there is a relatively small df, but the computation time still demonstrates the magnitude between the different methods can make.  

You can use the Python37 env. 

In [1]:
import os, sys
from os.path import join, exists, basename, splitext, dirname
import pandas as pd    

In [2]:
test_file = '/home/svc_jupyter/tutorials/data/speedup_testdata.txt'

df = pd.read_csv(test_file, names = ["ReadID", "Length", "Positions", "String"], sep = "\t")

# Using df.iterrows is slow

In [4]:
%%time

all_poses =  list()
all_bc_orients =  list()
units = list()

# ---------------------------------------------------------------------#
for index, row in df.iterrows(): # <------ This is where the issue lies. 
# ---------------------------------------------------------------------#

    motif_line = row['String']
    pos_line = row['Positions']
    
    motifs = motif_line.split(":")
    poses = pos_line.split(":")
    
    bc_orients = list() 
    bc_poses = list()
    

    for i in range(len(motifs)):
        motif = motifs[i]
        
        if motif in ['-BC_SV40','+BC_SV40']:
        
            bc_orient = motif[0]
            bc_orients.append(bc_orient)
        
            pos = poses[i]
            bc_poses.append(pos)
        else:
            continue
        
    all_bc_orients.append(':'.join(bc_orients))
    all_poses.append(':'.join(bc_poses))
    
    num_unit = len(bc_orients)
    units.append(num_unit)

new_data = [all_poses,all_bc_orients,units]
new_data_df = pd.DataFrame(data={'Orientation':all_bc_orients,'Pos':all_poses,'Units':units})
new_data_df.head()

CPU times: user 32.9 s, sys: 93.7 ms, total: 33 s
Wall time: 33.1 s


,Orientation,Pos,Units
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


## first let's functionalize the inside loop

both below functions have similar performance, use what is most intuitive

my preference: list comprehension

In [5]:
def process_barcodes_lc(motif_line, pos_line):
    motifs = motif_line.split(":")
    poses = pos_line.split(":")
    bc_and_pos = [(motif,pos) for motif, pos in zip(motifs,poses) if motif in ['-BC_SV40','+BC_SV40']]
    bc_orient = ':'.join([motif[0] for (motif,pos) in bc_and_pos])
    bc_pos = ':'.join([pos for (motif,pos) in bc_and_pos])
    units = len(bc_and_pos)
    return([bc_orient,bc_pos,units])

copy/paste of foor loop

In [6]:
def process_barcodes_floop(motif_line, pos_line):
    motifs = motif_line.split(":")
    poses = pos_line.split(":")
    
    bc_orients = list() 
    bc_poses = list()
    
    for i in range(len(motifs)):
        motif = motifs[i]
        
        if motif in ['-BC_SV40','+BC_SV40']:
            bc_orient = motif[0]
            bc_orients.append(bc_orient)
            
            pos = poses[i]
            bc_poses.append(pos)
        else:
            continue
    num_unit = len(bc_orients)
    bc_orients = ':'.join(bc_orients)
    bc_poses = ':'.join(bc_poses)
    return([bc_orients,bc_poses,num_unit])

### demonstartion of list comprehension function

In [7]:
%%time

all_poses =  list()
all_bc_orients =  list()
units = list()

for index, row in df.iterrows():

    motif_line = row['String']
    pos_line = row['Positions']
    
    bc_orients, bc_poses, num_unit = process_barcodes_lc(motif_line, pos_line)
        
    all_bc_orients.append(bc_orients)
    all_poses.append(bc_poses)
    
    units.append(num_unit)

new_data = [all_poses,all_bc_orients,units]
new_data_df = pd.DataFrame(data={'Orientation':all_bc_orients,'Pos':all_poses,'Units':units})
new_data_df.head()

CPU times: user 30 s, sys: 90 ms, total: 30.1 s
Wall time: 30.2 s


,Orientation,Pos,Units
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


### demonstartion of for loop function

In [8]:
%%time

all_poses =  list()
all_bc_orients =  list()
units = list()

for index, row in df.iterrows():

    motif_line = row['String']
    pos_line = row['Positions']
    
    bc_orients, bc_poses, num_unit = process_barcodes_floop(motif_line, pos_line)
        
    all_bc_orients.append(bc_orients)
    all_poses.append(bc_poses)
    
    units.append(num_unit)
    
new_data = [all_poses,all_bc_orients,units]
new_data_df = pd.DataFrame(data={'Orientation':all_bc_orients,'Pos':all_poses,'Units':units})
new_data_df.head()

CPU times: user 32 s, sys: 82.2 ms, total: 32 s
Wall time: 32.1 s


,Orientation,Pos,Units
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


**no speedup yet...**

## Alternative 1 to df.iterrows: df.apply

### Using the apply function and expanding to df

In [14]:
%%time
# ---------- DO NOT USE EXPAND ---------------- #
new_data_df = df.apply(lambda x: process_barcodes_lc(x['String'],x['Positions']), axis=1, result_type='expand')

new_data_df.head()
# ---------- DO NOT USE EXPAND ---------------- #

CPU times: user 53.7 s, sys: 241 ms, total: 54 s
Wall time: 54.1 s


,0,1,2
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


Using the **expand** function on a large df is **VERY** time consuming !!!! Worse than the original for loop !!!!

### faster: Using the apply function but reconstructing the dataframe

In [10]:
%%time
new_data = df.apply(lambda x: process_barcodes_lc(x['String'],x['Positions']), axis=1)
new_data = new_data.to_list() 

new_data_df = pd.DataFrame(new_data, columns = ['Orientation','Pos','Units'])
new_data_df.head()

CPU times: user 8.96 s, sys: 31.8 ms, total: 9 s
Wall time: 9.01 s


,Orientation,Pos,Units
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


The data from apply comes out as a series, by converting to a list we can then convert to a df using pandas standard "pd.DataFrame" function. 

The reasoning this is faster is because in expand, the apply function is trying to concat a dataframe for each row, so if you have 10'000s or 100'000s of rows, you are calling the it is more expensive then dealing with python native function many times. 

If we do not use the expand option, this gives us a 3x-4x speedup already. 


## Alternative 2 to df.iterrows: pandas-free operation

### Using list comprehension

In [11]:
%%time
new_data = [process_barcodes_lc(motif_line, pos_line) for motif_line, pos_line in zip(df['String'],df['Positions'])] 

new_data_df = pd.DataFrame(new_data, columns = ['Orientation','Pos','Units'])
new_data_df.head()

CPU times: user 1.44 s, sys: 17 ms, total: 1.46 s
Wall time: 1.46 s


,Orientation,Pos,Units
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


fastest time yet

#### But what if a 1-liner it is getting too hard to understand?

No problem! A regular a for loop works just as quickly. 

In [12]:
%%time

new_data = []
string_data = df['String'].to_list()
pos_data = df['Positions'].to_list()

for i in range(len(df)):
    motif_line = string_data[i]
    pos_line  = pos_data[i]
    new_data.append(process_barcodes_lc(motif_line, pos_line))
    
new_data_df = pd.DataFrame(new_data, columns = ['Orientation','Pos','Units'])
new_data_df.head()

CPU times: user 1.71 s, sys: 4.06 ms, total: 1.71 s
Wall time: 1.71 s


,Orientation,Pos,Units
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


The speedup in python comes from working with lists indead of with pandas objects.  
The 1-liners can help simplify the code you 
need to remember, but also can affect understanding of the code. 

## Easy mistake to make!

In [13]:
%%time

new_data = []
string_data = df['String'] # <------- MISSING CONVERSION TO LIST 
pos_data = df['Positions'] # <------- MISSING CONVERSION TO LIST
for i in range(len(df)):
    motif_line = string_data[i]
    pos_line  = pos_data[i]
    new_data.append(process_barcodes_lc(motif_line, pos_line))
    
new_data_df = pd.DataFrame(new_data, columns = ['Orientation','Pos','Units'])
new_data_df.head()

CPU times: user 8.99 s, sys: 12 ms, total: 9 s
Wall time: 9.02 s


,Orientation,Pos,Units
0,-,3462-3685,1
1,,,0
2,-,1104-1353,1
3,+,59-311,1
4,+,2766-3021,1


Here we can see the difference between the first for loop, 30 seconds, and the fastest, about 1 second, is around 30-fold, however these differences usually scale exponentially with dataframe size and can be the difference between minutes and days. 

Let me know if you have any questions!